# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import Markdown, display

import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.widgets.mermaid_viz as mmw
import edurel.widgets.chatman as cmw
import edurel.utils.misc as mu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [ ]:
con = ddbu.con_file(f"{DB_DIR}/adw-olap-small/adw-etl.duckdb", read_only=False)
# ddbu.exe_sql_file(con, f"{DB_DIR}/adw-olap-small/schema.sql")
# ddbu.schema_yaml_print(con,additional_fks={})

In [8]:
con.close()

# ETL

## Currency

In [ ]:
sql = """
INSERT INTO DimCurrency (CurrencyKey, CurrencyName)
SELECT
    ROW_NUMBER() OVER (ORDER BY CurrencyCode) AS CurrencyKey,
    Name AS CurrencyName
FROM Currency;
"""
ddbu.exe_sql(con, sql)

None


In [7]:
sql = """
select * from DimCurrency limit 5;
"""
ddbu.sql_print(con, sql)

┌─────────────┬───────────────────────────────┐
│ CurrencyKey │         CurrencyName          │
│    int32    │            varchar            │
├─────────────┼───────────────────────────────┤
│           1 │ Emirati Dirham                │
│           2 │ Afghani                       │
│           3 │ Lek                           │
│           4 │ Armenian Dram                 │
│           5 │ Netherlands Antillian Guilder │
└─────────────┴───────────────────────────────┘



In [ ]:
sql = """

"""
ddbu.exe_sql(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)

In [ ]:
sql = """

"""
ddbu.sql_print(con, sql)